<a href="https://colab.research.google.com/github/Sangyups/CharRNN/blob/main/CharRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

KeyboardInterrupt: ignored

In [ ]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [ ]:
girl_name = pd.read_csv("girls_name.csv")
boy_name = pd.read_csv("boys_name.csv")

In [ ]:
print(max(len(i) for i in girl_name['name']))
print(max(len(i) for i in boy_name['name']))

10
11


In [ ]:
girl_name.head()

,name
0,Olivia
1,Emma
2,Ava
3,Sophia
4,Isabella


In [ ]:
boy_name.head()

,name
0,Liam
1,Noah
2,Oliver
3,Elijah
4,William


In [ ]:
girl_name['gender'] = [0] * len(girl_name)
boy_name['gender'] = [1] * len(boy_name)

In [ ]:
girl_name.head()

,name,gender
0,Olivia,0
1,Emma,0
2,Ava,0
3,Sophia,0
4,Isabella,0


In [ ]:
boy_name.head()

,name,gender
0,Liam,1
1,Noah,1
2,Oliver,1
3,Elijah,1
4,William,1


In [ ]:
data = pd.concat([girl_name, boy_name])

In [ ]:
class NameDataset(Dataset):
    def __init__(self, data):
        super().__init__()
        self.data = data.sample(frac=1).reset_index(drop=True)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        a = self.data['name'].loc[idx].lower()
        n = np.array([np.eye(26)[ord(x)-0x61] for x in a])
        while len(n) < 12:
            n = np.append(n, [[0]*26], axis=0)

        y = self.data['gender'].loc[idx]

        return torch.Tensor(n), torch.FloatTensor([y])



In [ ]:
dataset = NameDataset(data)
dataloader = DataLoader(dataset=dataset, batch_size=16, shuffle=True, num_workers=2)

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = nn.RNN(input_size=26, hidden_size=32, num_layers=1)
        self.linear = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        out = self.rnn(x)[0][:, -1, :]
        return self.sigmoid(self.linear(out))

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
model = Model().to(device)
loss_func = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

cuda


In [ ]:
total_epochs = 20
for epoch in range(total_epochs):
    success = 0
    loss_tot = []
    for x, y in dataloader:
        x = x.to(device)
        y_pred = model(x)
        y = y.to(device)
        loss = loss_func(y_pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        y_pred[y_pred >= 0.5] = 1
        y_pred[y_pred < 0.5] = 0
        success += (y_pred == y).sum()

        loss_tot.append(loss.item())

    print(f'Epoch: {epoch}, Train Loss: {np.array(loss_tot).mean():.3f}, Train Accuracy: {success/2000:.3f}')

Epoch: 0, Train Loss: 0.694, Train Accuracy: 0.485
Epoch: 1, Train Loss: 0.694, Train Accuracy: 0.490
Epoch: 2, Train Loss: 0.694, Train Accuracy: 0.493
Epoch: 3, Train Loss: 0.694, Train Accuracy: 0.498
Epoch: 4, Train Loss: 0.693, Train Accuracy: 0.508
Epoch: 5, Train Loss: 0.694, Train Accuracy: 0.496
Epoch: 6, Train Loss: 0.694, Train Accuracy: 0.496
Epoch: 7, Train Loss: 0.693, Train Accuracy: 0.499
Epoch: 8, Train Loss: 0.693, Train Accuracy: 0.492
Epoch: 9, Train Loss: 0.694, Train Accuracy: 0.498
Epoch: 10, Train Loss: 0.693, Train Accuracy: 0.504
Epoch: 11, Train Loss: 0.694, Train Accuracy: 0.487
Epoch: 12, Train Loss: 0.694, Train Accuracy: 0.479
Epoch: 13, Train Loss: 0.694, Train Accuracy: 0.493
Epoch: 14, Train Loss: 0.693, Train Accuracy: 0.483
Epoch: 15, Train Loss: 0.694, Train Accuracy: 0.486
Epoch: 16, Train Loss: 0.693, Train Accuracy: 0.500
Epoch: 17, Train Loss: 0.694, Train Accuracy: 0.498
Epoch: 18, Train Loss: 0.693, Train Accuracy: 0.489
Epoch: 19, Train Loss:

In [ ]:
def predict(name):
    a = name.lower()
    n= np.array([[np.eye(26)[ord(x)-0x61] for x in a]])
    while len(n[0]) < 12:
        n = np.append(n, np.array([[0]*26]).reshape(1, 1, 26), axis=1)
    x = torch.Tensor(n)
    y_pred = model(x.to(device))
    if y_pred < 0.5:
        print(f'{name} is a girl')
    else:
        print(f'{name} is a boy')

In [ ]:
predict('emma')

emma is a boy
